# Supervised Machine Learning Project 
## Asset Maintenance use-case for a Synthetic Railway Company
#### 27.04.2022

</br>


### Data Set Information:

- Asset dataset - This is data that describes the asset
- Asset repairs - This details the repairs of assets
- Asset replacements - This is a list of times when assets have been replaced (failed)
- Planned maintenance - This describes if the reapir or replacement 
- Additional asset attributes - This gives additional infomation about assets


### References: 
 

In [88]:
# importing libraries
import os
import numpy as np
import pandas as pd

In [89]:
# dataset files location
folder_path = './SyntheticData'

In [90]:
# get all files
files = os.listdir(folder_path)
print(files)

# organize all files in a dictionary of dataframes
df_dict = {}
filenames = []
for file in files: 
    filenames.append(file[:-4])  # slice .csv extension from name
    df_dict[file[:-4]] = pd.read_csv(folder_path + '/' + file)    

['asset_attribute_data_general.csv', 'asset_attribute_data_usage.csv', 'asset_attribute_data_weather.csv', 'asset_data.csv', 'planned_data.csv', 'repair_data.csv', 'replacement_data.csv']


# 1. Getting familiar with the data

In [91]:
df_dict['asset_attribute_data_general'].head()

,asset_id,asset_install_team,asset_line,asset_material
0,A:cpjwvv,team4,east_line,iron
1,A:xoauw0,team3,south_line,steel
2,A:xpqps2,team1,east_line,iron
3,A:m44bmt,team4,west_line,steel
4,A:rwkyh1,team2,north_line,steel


In [92]:
df_dict['asset_attribute_data_usage'].head()

,asset_id,asset_trains_per_hour
0,A:cpjwvv,2.0
1,A:xoauw0,4.0
2,A:xpqps2,4.0
3,A:m44bmt,4.0
4,A:rwkyh1,1.0


In [93]:
df_dict['asset_attribute_data_weather'].head()

,asset_id,asset_weather_cluster
0,A:cpjwvv,standard
1,A:xoauw0,standard
2,A:xpqps2,sun
3,A:m44bmt,sun
4,A:rwkyh1,standard


In [94]:
df_dict['asset_data'].head()

,asset_id,end_date,latitude,longitude,previous_repairs,previous_unplanned,start_date
0,A:cpjwvv,2012-02-26,51.108966,-1.382032,0,0,2008-06-26
1,A:xoauw0,2005-03-01,58.369692,-4.982743,0,0,2003-06-08
2,A:xpqps2,2009-10-27,55.759249,-4.593591,0,0,2007-03-03
3,A:m44bmt,2008-02-23,52.289685,-1.418834,0,0,2007-01-17
4,A:rwkyh1,2013-06-21,55.686979,-4.542553,0,0,2012-06-03


The previous/above tables all share an identification column that could be used to merge them: <b> asset_id </b>

In [95]:
df_dict['planned_data'].head()

,asset_id,event_date,event_id,planned
0,A:cpjwvv,2011-05-06,E:6avbyn53sz,False
1,A:cpjwvv,2008-08-30,E:trj6ixh0o9,False
2,A:cpjwvv,2009-02-06,E:y5bjtvl55l,False
3,A:cpjwvv,2010-02-07,E:evsbi3vj21,False
4,A:cpjwvv,2010-02-17,E:njcddphr3b,False


In [96]:
df_dict['repair_data'].head()

,asset_id,event_date,event_id,installed_date,previous_repairs,previous_unplanned
0,A:cpjwvv,2011-05-06,E:6avbyn53sz,2011-03-25,1,1
1,A:xoauw0,2004-05-10,E:8h4dpsljy3,2003-08-28,1,1
2,A:xoauw0,2004-10-18,E:zl0sp2rps7,2004-05-10,2,2
3,A:xpqps2,2009-09-07,E:ndxhmxamfi,2007-10-03,1,1
4,A:m44bmt,2007-03-29,E:01zkvga68v,2007-03-26,1,1


In [97]:
df_dict['replacement_data'].head()

,asset_id,event_date,event_id,installed_date,previous_repairs,previous_unplanned
0,A:cpjwvv,2008-08-30,E:trj6ixh0o9,2008-06-26,0,0
1,A:cpjwvv,2009-02-06,E:y5bjtvl55l,2008-08-30,0,0
2,A:cpjwvv,2010-02-07,E:evsbi3vj21,2009-02-06,0,0
3,A:cpjwvv,2010-02-17,E:njcddphr3b,2010-02-07,0,0
4,A:cpjwvv,2010-05-18,E:fry99yixfk,2010-02-17,0,0


### Some text here...

In [98]:
# Check datasets size
for name in filenames:
    print(name, df_dict[name].shape)

asset_attribute_data_general (200, 4)
asset_attribute_data_usage (200, 2)
asset_attribute_data_weather (200, 2)
asset_data (200, 7)
planned_data (2032, 4)
repair_data (859, 6)
replacement_data (1173, 6)


### Merging the datasets

In [99]:
df_assets = pd.merge(df_dict['asset_attribute_data_general'], df_dict['asset_attribute_data_usage'], on='asset_id')
df_assets = pd.merge(df_assets, df_dict['asset_attribute_data_weather'], on='asset_id')

In [100]:
df_assets

,asset_id,asset_install_team,asset_line,asset_material,asset_trains_per_hour,asset_weather_cluster
0,A:cpjwvv,team4,east_line,iron,2.0,standard
1,A:xoauw0,team3,south_line,steel,4.0,standard
2,A:xpqps2,team1,east_line,iron,4.0,sun
3,A:m44bmt,team4,west_line,steel,4.0,sun
4,A:rwkyh1,team2,north_line,steel,1.0,standard
...,...,...,...,...,...,...
195,A:271d3q,team4,south_line,iron,4.0,standard
196,A:6tjk91,team3,west_line,iron,4.0,standard
197,A:j9z5oa,team2,east_line,iron,8.0,heavy_rain
198,A:100egl,team2,west_line,alloy,2.0,sun


In [101]:
df_assets = pd.merge(df_assets, df_dict['asset_data'], on='asset_id')

In [102]:
df_assets

,asset_id,asset_install_team,asset_line,asset_material,asset_trains_per_hour,asset_weather_cluster,end_date,latitude,longitude,previous_repairs,previous_unplanned,start_date
0,A:cpjwvv,team4,east_line,iron,2.0,standard,2012-02-26,51.108966,-1.382032,0,0,2008-06-26
1,A:xoauw0,team3,south_line,steel,4.0,standard,2005-03-01,58.369692,-4.982743,0,0,2003-06-08
2,A:xpqps2,team1,east_line,iron,4.0,sun,2009-10-27,55.759249,-4.593591,0,0,2007-03-03
3,A:m44bmt,team4,west_line,steel,4.0,sun,2008-02-23,52.289685,-1.418834,0,0,2007-01-17
4,A:rwkyh1,team2,north_line,steel,1.0,standard,2013-06-21,55.686979,-4.542553,0,0,2012-06-03
...,...,...,...,...,...,...,...,...,...,...,...,...
195,A:271d3q,team4,south_line,iron,4.0,standard,2012-05-12,52.883360,-2.048643,0,0,2005-10-15
196,A:6tjk91,team3,west_line,iron,4.0,standard,2016-07-08,54.765397,-2.104487,0,0,2013-08-22
197,A:j9z5oa,team2,east_line,iron,8.0,heavy_rain,2017-06-01,54.332715,-2.370033,0,0,2013-05-21
198,A:100egl,team2,west_line,alloy,2.0,sun,2013-08-18,54.151059,-2.068212,0,0,2008-05-04


In [103]:
# Remove latitude and longitude features. Realy hard to work with coordinates.
df_assets = df_assets.drop(['latitude', 'longitude'], axis=1)

In [104]:
# Just organizing the columns
def move_col(name, pos):
    column = df_assets.pop(name)
    df_assets.insert(pos, name, column)
    
move_col('start_date', 1)
move_col('end_date', 2)

In [105]:
df_assets

,asset_id,start_date,end_date,asset_install_team,asset_line,asset_material,asset_trains_per_hour,asset_weather_cluster,previous_repairs,previous_unplanned
0,A:cpjwvv,2008-06-26,2012-02-26,team4,east_line,iron,2.0,standard,0,0
1,A:xoauw0,2003-06-08,2005-03-01,team3,south_line,steel,4.0,standard,0,0
2,A:xpqps2,2007-03-03,2009-10-27,team1,east_line,iron,4.0,sun,0,0
3,A:m44bmt,2007-01-17,2008-02-23,team4,west_line,steel,4.0,sun,0,0
4,A:rwkyh1,2012-06-03,2013-06-21,team2,north_line,steel,1.0,standard,0,0
...,...,...,...,...,...,...,...,...,...,...
195,A:271d3q,2005-10-15,2012-05-12,team4,south_line,iron,4.0,standard,0,0
196,A:6tjk91,2013-08-22,2016-07-08,team3,west_line,iron,4.0,standard,0,0
197,A:j9z5oa,2013-05-21,2017-06-01,team2,east_line,iron,8.0,heavy_rain,0,0
198,A:100egl,2008-05-04,2013-08-18,team2,west_line,alloy,2.0,sun,0,0


### Merging with the dataset including event_id feature

In [106]:
df_repairs = pd.merge(
    df_dict['repair_data'], 
    df_dict['replacement_data'], 
    on=['asset_id', 'event_id', 'event_date', 'installed_date', 'previous_repairs', 'previous_unplanned'], 
    how='outer')

In [107]:
df_repairs

,asset_id,event_date,event_id,installed_date,previous_repairs,previous_unplanned
0,A:cpjwvv,2011-05-06,E:6avbyn53sz,2011-03-25,1,1
1,A:xoauw0,2004-05-10,E:8h4dpsljy3,2003-08-28,1,1
2,A:xoauw0,2004-10-18,E:zl0sp2rps7,2004-05-10,2,2
3,A:xpqps2,2009-09-07,E:ndxhmxamfi,2007-10-03,1,1
4,A:m44bmt,2007-03-29,E:01zkvga68v,2007-03-26,1,1
...,...,...,...,...,...,...
2027,A:bo36ea,2008-03-11,E:tiimfa24q1,2007-12-13,0,0
2028,A:bo36ea,2008-06-08,E:o7x8b82wrs,2008-05-26,0,0
2029,A:bo36ea,2008-06-08,E:urquo998rq,2008-06-08,0,0
2030,A:bo36ea,2008-12-12,E:cxh6sozhrx,2008-06-08,0,0


In [108]:
df_repairs = pd.merge(
    df_repairs, 
    df_dict['planned_data'], 
    on=['asset_id', 'event_id', 'event_date'])

In [109]:
df_repairs

,asset_id,event_date,event_id,installed_date,previous_repairs,previous_unplanned,planned
0,A:cpjwvv,2011-05-06,E:6avbyn53sz,2011-03-25,1,1,False
1,A:xoauw0,2004-05-10,E:8h4dpsljy3,2003-08-28,1,1,False
2,A:xoauw0,2004-10-18,E:zl0sp2rps7,2004-05-10,2,2,False
3,A:xpqps2,2009-09-07,E:ndxhmxamfi,2007-10-03,1,1,False
4,A:m44bmt,2007-03-29,E:01zkvga68v,2007-03-26,1,1,False
...,...,...,...,...,...,...,...
2027,A:bo36ea,2008-03-11,E:tiimfa24q1,2007-12-13,0,0,False
2028,A:bo36ea,2008-06-08,E:o7x8b82wrs,2008-05-26,0,0,False
2029,A:bo36ea,2008-06-08,E:urquo998rq,2008-06-08,0,0,True
2030,A:bo36ea,2008-12-12,E:cxh6sozhrx,2008-06-08,0,0,False


#### Comparing assets info with repairements/replacements

In [110]:
df_repairs.loc[df_repairs['asset_id'] == 'A:cpjwvv', :]

,asset_id,event_date,event_id,installed_date,previous_repairs,previous_unplanned,planned
0,A:cpjwvv,2011-05-06,E:6avbyn53sz,2011-03-25,1,1,False
859,A:cpjwvv,2008-08-30,E:trj6ixh0o9,2008-06-26,0,0,False
860,A:cpjwvv,2009-02-06,E:y5bjtvl55l,2008-08-30,0,0,False
861,A:cpjwvv,2010-02-07,E:evsbi3vj21,2009-02-06,0,0,False
862,A:cpjwvv,2010-02-17,E:njcddphr3b,2010-02-07,0,0,False
863,A:cpjwvv,2010-05-18,E:fry99yixfk,2010-02-17,0,0,False
864,A:cpjwvv,2010-06-03,E:eku0zyi4cd,2010-05-18,0,0,False
865,A:cpjwvv,2010-08-06,E:sv5orf2kc2,2010-06-03,0,0,False
866,A:cpjwvv,2010-09-01,E:giwnv8wxjw,2010-08-06,0,0,False
867,A:cpjwvv,2010-12-19,E:a8iikd6tuj,2010-09-01,0,0,False


In [111]:
df_assets.loc[df_assets['asset_id'] == 'A:cpjwvv', :]

,asset_id,start_date,end_date,asset_install_team,asset_line,asset_material,asset_trains_per_hour,asset_weather_cluster,previous_repairs,previous_unplanned
0,A:cpjwvv,2008-06-26,2012-02-26,team4,east_line,iron,2.0,standard,0,0


* Start date corresponds to the earlier date of the installed date;
* When there is an event, the date will count as the installed date for next event
* What is the end date used for?

In [112]:
df_repairs.loc[df_repairs['asset_id'] == 'A:m44bmt', :]

,asset_id,event_date,event_id,installed_date,previous_repairs,previous_unplanned,planned
4,A:m44bmt,2007-03-29,E:01zkvga68v,2007-03-26,1,1,False
5,A:m44bmt,2008-01-22,E:og70gj9mu2,2007-11-07,1,1,False
877,A:m44bmt,2007-03-26,E:f7vew1qt43,2007-01-17,0,0,False
878,A:m44bmt,2007-10-08,E:6bdckn1ae6,2007-03-29,0,0,False
879,A:m44bmt,2007-11-07,E:2woc3z6iqu,2007-10-08,0,0,False


In [113]:
df_assets.loc[df_assets['asset_id'] == 'A:m44bmt', :]

,asset_id,start_date,end_date,asset_install_team,asset_line,asset_material,asset_trains_per_hour,asset_weather_cluster,previous_repairs,previous_unplanned
3,A:m44bmt,2007-01-17,2008-02-23,team4,west_line,steel,4.0,sun,0,0


In [120]:
df_repairs['planned'].value_counts()

False    1743
True      289
Name: planned, dtype: int64